In [10]:
# ============================================================================
# This code is part of Assignment 3 of CS561 (Executive M-Tech -AI Assignment-3)
# Submitted by:
#        IITP001300: Sukhvinder Singh  (email id: sukhvinder.malik13@gmail.com)
#        IITP001316: Manjit Singh Duhan (email id: duhan.manjit@gmail.com)
#        IITP001508: Atul Singh (email id: atulsingh.xcvi@gmail.com)
#============================================================================

In [1]:
import sys
import numpy as np
import heapq
from datetime import datetime

In [2]:
class node_state():
    def __init__(self, matrix, parent=None, move=None, goal=None):
        self.matrix      = np.array(matrix)
        self.parent      = parent           # The parent to this state. The start state will not have any parrent
        self.goal        = goal
        self.move        = move             # What move (Left/Right/Up/Down) has led to this state
        self.max_row     = len(matrix)      # Max rows in the matrix
        self.max_column  = len(matrix[0])   # Max columns in the matrix
        self.hash        = self.get_hash()  # Has of the matrix. Will be used while comparing states.
        self.depth       = self.set_depth() # The depth of this state from the state_state
        self.actual_cost = self.set_actual_cost()  # g(n) i.e. actual cost of this state from the start state
        
        # initializing with 0. It's responsibility of caller to set the cost based on desired heuristic function before using this
        self.cost = 0 
    
    def shape(self):
        return self.max_row, self.max_column
        
    def get_value(self, i, j):
        ''' Get the value from mattrix at [i][j] location '''
        return self.matrix[i, j]
    
    def get_i_j_of(self, val):
        '''Get the i & j of the value in the matrix '''
        return np.where(self.matrix == val)
    
    def get_copy(self):
        ''' Returns the copy of current matrix '''
        return self.matrix.copy()
    
    def set_depth(self):
        ''' Set the depth of this state. The depth will be parent's depth + 1 '''
        if self.parent is not None:
            return self.parent.get_depth() + 1
        else:
            return 0

    def set_actual_cost(self):
        ''' Set the cost i.e. g(n) of this state from the start state.
            As each step is having cost of 1 so we are fixing to to parent cost + 1'''
        if self.parent is not None:
            return self.parent.actual_cost + 1
        else:
            return 0
    
    def get_cost(self):
        return self.cost
        
    def get_depth(self):
        ''' Get function to get the depth of the state '''
        return self.depth

    def __lt__(self, other):
        return self.cost < other.cost
    
    def get_hash(self):
        '''
        Let us create the hash of the matrix so that the we will add the hash of explored state
        in the explored list. Before processing any state, we will check if that state us already
        explored or not.
        To calculate the hash, we are makeing int of the matrix as below
        [[123]
         [456]
         [780]] = 1234567890
        So, every matrix will have a unique hash value
        '''
        multiplier = 100000000
        value = 0
        for i in range(self.max_row):
            for j in range (self.max_column):
                value += self.matrix[i, j] * multiplier
                multiplier //= 10
        return value
    
    def print(self):
        ''' print the matrix '''
        for i in range (self.max_row):
            for j in range (self.max_column):
                val = self.matrix[i, j]
                if val == 0:
                    print("B", end=" ")
                else:
                    print(val, end=" ")
            print(" ")
    
    
    def get_Heuristics_1(self):
        my_cost = 0
        for i in range(self.max_row):
            for j in range(self.max_column):
                #if the current tile is displaced then increment the value
                if self.get_value(i, j) != self.goal.get_value(i, j):
                    my_cost += 1
        return my_cost

    
    def get_Heuristics_2(self):
        my_cost = 0
        for i in range(self.max_row):
            for j in range(self.max_column):
                #Get the value of tile at i, j
                val = self.get_value(i, j)
                
                # Get the value of i, j for the value in goal state
                goal_i, goal_j = self.goal.get_i_j_of(val)
                
                #Manhattan distance = |current_row - goal_row| + |current_column - goal_column|
                distance = abs(i - goal_i) + abs(j - goal_j)
                
                # sum up the Manhattan distance
                my_cost += distance
        return int(my_cost)
    
    def get_heuristic_cost(self, use_heuristic_function):
        
        if(use_heuristic_function == 1):
            return self.get_Heuristics_1()
        elif(use_heuristic_function == 2):
            return self.get_Heuristics_2()
        else:
            raise Exception("This is not a supported heuristic function type")
            return 0
        
    def set_cost(self, heuristic_function):
        a = self.actual_cost
        b = self.get_heuristic_cost(heuristic_function)
        self.cost = b


In [3]:
class puzzle():
    def __init__(self, start, goal):
        # Check if the nput Start state and Goal state are valid or not.
        self.validate_input_matrix(start)
        self.validate_input_matrix(goal)
        
        self.goal_state    = node_state(goal)       
        self.start_state   = node_state(start, goal=self.goal_state)
        
        self.max_row, self.max_column = start.shape
        self.close_list = []   # This list will contain the states that are explored at any moment
        self.solution_steps = []  # This list is used to fill while doing the back-tracing
        self.start_time     = datetime.now()
        self.end_time       = datetime.now()
        self.optimal_path_cost = 0    # sum of g(n) of all states
        self.total_of_fn = 0          # sum of f(n) of all states
        self.number_of_states_to_optimal_path = 0
        self.states_explored = 0  # the number of states explored at any moment of time.        


    def validate_input_matrix(self, matrix):
        ''' Validate the matrix. The matrix should have 0-8 values and no value should be repeated.
            While creating the puzzle object, validation should be done on start_state & goal_state '''
        max_row = len(matrix)
        max_column = len(matrix[0])
        
        for value in range(0, max_row*max_column):
            found = False;
            for i in range(max_row):
                for j in range(max_column):
                    if(value == matrix[i, j]):
                        found = True
            
            if(found == False):
                raise Exception("This is not a valid matrix")

        
    def get_neighbor_states(self, current_state):
        ''' Get the neighbors of the current state. The neighbors will be the states that comes
            after taking Left/Right/Up/Down move.
            No validation is done on the neighbor in this function.
            The called of this function need to do all the required validations'''
        
        move_directions = [("LEFT", 0, -1), ("RIGHT", 0, 1), ("UP", -1, 0), ("DOWN", 1, 0)]
        row, col = current_state.get_i_j_of(B)
        neighbors = []
        
        for move, i_offset, j_offset in move_directions:
            new_i = row + i_offset
            new_j = col + j_offset
            
            # move the blank space Left/Right/Up/Down if it is not crossing the boundries
            if ( 0 <= new_i < self.max_row ) and ( 0 <= new_j < self.max_column) :
                new_state = self.move(current_state, move, row[0], col[0])
                neighbors.append(new_state)

        return neighbors


    def move(self, current_state, move, i, j):
        '''MOve the blank space to Left/Right/Up/Down and return the new state
           The current state will be parent to tthis new state'''
        temp_matrix = current_state.get_copy()
        
        # Store the value of matrix[i, j] in temp varianble. This will be used during swap operation
        temp_val = temp_matrix[i, j]
        
        if move == "RIGHT":
            temp_matrix[i, j] = temp_matrix[i, j+1]
            temp_matrix[i, j+1] = temp_val
        elif move == "LEFT":
            temp_matrix[i, j] = temp_matrix[i, j-1]
            temp_matrix[i, j-1] = temp_val
        elif move == "UP":
            temp_matrix[i, j] = temp_matrix[i-1, j]
            temp_matrix[i-1, j] = temp_val
        elif move == "DOWN":
            temp_matrix[i, j] = temp_matrix[i+1, j]
            temp_matrix[i+1, j] = temp_val

        # Create the state from this new matrix. Assign the current matrix as parent to this new state
        new_state = node_state(temp_matrix, current_state, move, self.goal_state)
        return new_state

    def print_solution(self):
        ''' Print the solution '''

        print("\n---------------------")

        while len(self.solution_steps) != 0:
            current_state = self.solution_steps.pop(0)
            print("\nMove: ", current_state.move)
            current_state.print()
    
    def solve(self, current_state, heuristic_function):
   
        # Keep looping until solution is not found
        while True:

            self.states_explored += 1

            # if hash of current state and Goal state is same then we have reached the goal :D
            if current_state.hash == self.goal_state.hash:
                self.end_time = datetime.now()

                item = current_state
                while item.parent is not None:
                    self.solution_steps.insert(0, item)
                    
                    self.number_of_states_to_optimal_path += 1
                    self.optimal_path_cost += item.actual_cost
                    self.total_of_fn += item.cost
                    
                    item = item.parent
                return True
            
            # get the valid neighbors of current state. Drop the neighbor that is already explored list i.e. close list
            neighbors = self.get_neighbor_states(current_state)
            local_best_found = False
            
            for neighbor in neighbors:
                neighbor.set_cost(heuristic_function)
                if neighbor.get_cost() < current_state.get_cost():
                    local_best_found = True
                    current_state = neighbor
            
            if local_best_found == False:
                return False
        
        #Goal state is not found
        return False
    
    def display_statistics(self, type_of_algorithm, solution_found, heuristic_function):
        
        print(f"start state: {self.start_state.hash}    goal state: {self.goal_state.hash}    using h({heuristic_function})")

        self.end_time = datetime.now()
        execution_time = self.end_time - self.start_time
        
        sol_found = " Yes" if solution_found == True else " No"
        print(f"\tSolution found: {sol_found}")   


        print(f"\tTotal number of states explored: {self.states_explored}")
        print(f"\tTime taken for execution : {execution_time}")
            
        if solution_found == True:
            print(f"\tTotal number of states to the optimal path: {self.number_of_states_to_optimal_path}")
            print(f"\tTotal cost g(n) =  {self.optimal_path_cost}")
            print(f"\tTotal cost f(n) =  {self.total_of_fn}")
            print("\tOptimal Path is: ")
            self.print_solution()
            

In [4]:
class hill_climbing_puzzle_solver(puzzle):
    
    def find_the_solution(self, heuristic_function):
        type_of_algorithm = "Hill Climbing search"

        # Clear the explored list, solution steps
        self.solution_steps.clear()

        self.start_state.set_cost(heuristic_function)
        
        self.start_time = datetime.now()
        solution_found = self.solve(self.start_state, heuristic_function)

        self.display_statistics(type_of_algorithm, solution_found, heuristic_function)

        return solution_found

In [6]:
B=0  # Just to make the matrix more readable as eyes will easily catch "B" compared to 0.


max_retry_count = 10000

def find_solution_using_hill_climbing(heuristic_function, start_node_state, goal_state):
    retry_count = 1
    h = hill_climbing_puzzle_solver(start_node_state, goal_state)
    result = h.find_the_solution(heuristic_function)
    
    while( (result == False) and (retry_count < max_retry_count)):
        
        #take a random state
        random_array = np.arange(9)
        np.random.shuffle(random_array)
        start_node_state = random_array.reshape(3, 3)

        h = hill_climbing_puzzle_solver(start_node_state, goal_state)
        print(f"\nRetry Count: {retry_count}")
        result = h.find_the_solution(heuristic_function)

        retry_count += 1


# start_state = random_array.reshape(3, 3)
start_state =np.array([
       [6, 7, 3],
       [8, 4, 2],
       [1, 0, 5]])


# The goal state. This should be the goal where we need to reach
goal_state= np.array([
       [1, 2, 3],
       [4, 5, 6],
       [7, 8, B]])



print(f"====================================================================================")
print(f"=                  Hill Climbing Using h(1)                                       ==")
print(f"====================================================================================")
find_solution_using_hill_climbing(1, start_state, goal_state)


=                  Hill Climbing Using h(1)                                       ==
start state: 673842105    goal state: 123456780    using h(1)
	Solution found:  No
	Total number of states explored: 2
	Time taken for execution : 0:00:00.000741

Retry Count: 1
start state: 308524167    goal state: 123456780    using h(1)
	Solution found:  No
	Total number of states explored: 3
	Time taken for execution : 0:00:00.000249

Retry Count: 2
start state: 75824361    goal state: 123456780    using h(1)
	Solution found:  No
	Total number of states explored: 1
	Time taken for execution : 0:00:00

Retry Count: 3
start state: 46825371    goal state: 123456780    using h(1)
	Solution found:  No
	Total number of states explored: 1
	Time taken for execution : 0:00:00.000876

Retry Count: 4
start state: 560472831    goal state: 123456780    using h(1)
	Solution found:  No
	Total number of states explored: 1
	Time taken for execution : 0:00:00

Retry Count: 5
start state: 124083576    goal state: 123

start state: 247560318    goal state: 123456780    using h(1)
	Solution found:  No
	Total number of states explored: 3
	Time taken for execution : 0:00:00.001977

Retry Count: 1245
start state: 847653120    goal state: 123456780    using h(1)
	Solution found:  No
	Total number of states explored: 1
	Time taken for execution : 0:00:00

Retry Count: 1246
start state: 651832047    goal state: 123456780    using h(1)
	Solution found:  No
	Total number of states explored: 1
	Time taken for execution : 0:00:00

Retry Count: 1247
start state: 637458201    goal state: 123456780    using h(1)
	Solution found:  No
	Total number of states explored: 2
	Time taken for execution : 0:00:00

Retry Count: 1248
start state: 24683517    goal state: 123456780    using h(1)
	Solution found:  No
	Total number of states explored: 1
	Time taken for execution : 0:00:00

Retry Count: 1249
start state: 53128467    goal state: 123456780    using h(1)
	Solution found:  No
	Total number of states explored: 3
	Time 


Retry Count: 2388
start state: 435028617    goal state: 123456780    using h(1)
	Solution found:  No
	Total number of states explored: 2
	Time taken for execution : 0:00:00

Retry Count: 2389
start state: 801523467    goal state: 123456780    using h(1)
	Solution found:  No
	Total number of states explored: 4
	Time taken for execution : 0:00:00

Retry Count: 2390
start state: 861573240    goal state: 123456780    using h(1)
	Solution found:  No
	Total number of states explored: 1
	Time taken for execution : 0:00:00

Retry Count: 2391
start state: 416205738    goal state: 123456780    using h(1)
	Solution found:  No
	Total number of states explored: 3
	Time taken for execution : 0:00:00

Retry Count: 2392
start state: 476135820    goal state: 123456780    using h(1)
	Solution found:  No
	Total number of states explored: 1
	Time taken for execution : 0:00:00

Retry Count: 2393
start state: 105647328    goal state: 123456780    using h(1)
	Solution found:  No
	Total number of states expl

start state: 851026473    goal state: 123456780    using h(1)
	Solution found:  No
	Total number of states explored: 4
	Time taken for execution : 0:00:00.002996

Retry Count: 3572
start state: 517026843    goal state: 123456780    using h(1)
	Solution found:  No
	Total number of states explored: 1
	Time taken for execution : 0:00:00

Retry Count: 3573
start state: 415637820    goal state: 123456780    using h(1)
	Solution found:  No
	Total number of states explored: 1
	Time taken for execution : 0:00:00

Retry Count: 3574
start state: 57132846    goal state: 123456780    using h(1)
	Solution found:  No
	Total number of states explored: 2
	Time taken for execution : 0:00:00

Retry Count: 3575
start state: 487603215    goal state: 123456780    using h(1)
	Solution found:  No
	Total number of states explored: 1
	Time taken for execution : 0:00:00

Retry Count: 3576
start state: 463152087    goal state: 123456780    using h(1)
	Solution found:  No
	Total number of states explored: 1
	Time

In [8]:
print(f"\n====================================================================================")
print(f"=                   Hill Climbing Using h(2)                                       ==")
print(f"====================================================================================")
find_solution_using_hill_climbing(2, start_state, goal_state)



=                   Hill Climbing Using h(2)                                       ==
start state: 673842105    goal state: 123456780    using h(2)
	Solution found:  No
	Total number of states explored: 2
	Time taken for execution : 0:00:00.000966

Retry Count: 1
start state: 427163805    goal state: 123456780    using h(2)
	Solution found:  No
	Total number of states explored: 2
	Time taken for execution : 0:00:00

Retry Count: 2
start state: 603712845    goal state: 123456780    using h(2)
	Solution found:  No
	Total number of states explored: 4
	Time taken for execution : 0:00:00.002639

Retry Count: 3
start state: 253046178    goal state: 123456780    using h(2)
	Solution found:  No
	Total number of states explored: 2
	Time taken for execution : 0:00:00.001081

Retry Count: 4
start state: 168573024    goal state: 123456780    using h(2)
	Solution found:  No
	Total number of states explored: 1
	Time taken for execution : 0:00:00.000945

Retry Count: 5
start state: 501248367    goal

start state: 653047821    goal state: 123456780    using h(2)
	Solution found:  No
	Total number of states explored: 4
	Time taken for execution : 0:00:00.003912

Retry Count: 343
start state: 241783056    goal state: 123456780    using h(2)
	Solution found:  No
	Total number of states explored: 1
	Time taken for execution : 0:00:00

Retry Count: 344
start state: 684013275    goal state: 123456780    using h(2)
	Solution found:  No
	Total number of states explored: 2
	Time taken for execution : 0:00:00.000722

Retry Count: 345
start state: 56128374    goal state: 123456780    using h(2)
	Solution found:  No
	Total number of states explored: 5
	Time taken for execution : 0:00:00.002683

Retry Count: 346
start state: 584027136    goal state: 123456780    using h(2)
	Solution found:  No
	Total number of states explored: 2
	Time taken for execution : 0:00:00.000517

Retry Count: 347
start state: 483257160    goal state: 123456780    using h(2)
	Solution found:  No
	Total number of states e

start state: 64372815    goal state: 123456780    using h(2)
	Solution found:  No
	Total number of states explored: 5
	Time taken for execution : 0:00:00.001084

Retry Count: 657
start state: 541738206    goal state: 123456780    using h(2)
	Solution found:  No
	Total number of states explored: 1
	Time taken for execution : 0:00:00

Retry Count: 658
start state: 805726431    goal state: 123456780    using h(2)
	Solution found:  No
	Total number of states explored: 2
	Time taken for execution : 0:00:00

Retry Count: 659
start state: 735182046    goal state: 123456780    using h(2)
	Solution found:  No
	Total number of states explored: 2
	Time taken for execution : 0:00:00.001182

Retry Count: 660
start state: 271456308    goal state: 123456780    using h(2)
	Solution found:  No
	Total number of states explored: 2
	Time taken for execution : 0:00:00.000994

Retry Count: 661
start state: 183025746    goal state: 123456780    using h(2)
	Solution found:  No
	Total number of states explored

start state: 458613072    goal state: 123456780    using h(2)
	Solution found:  No
	Total number of states explored: 3
	Time taken for execution : 0:00:00.000923

Retry Count: 1015
start state: 184702365    goal state: 123456780    using h(2)
	Solution found:  No
	Total number of states explored: 3
	Time taken for execution : 0:00:00.001000

Retry Count: 1016
start state: 135742068    goal state: 123456780    using h(2)
	Solution found:  No
	Total number of states explored: 1
	Time taken for execution : 0:00:00

Retry Count: 1017
start state: 613457802    goal state: 123456780    using h(2)
	Solution found:  No
	Total number of states explored: 2
	Time taken for execution : 0:00:00.001422

Retry Count: 1018
start state: 81672543    goal state: 123456780    using h(2)
	Solution found:  No
	Total number of states explored: 1
	Time taken for execution : 0:00:00.000341

Retry Count: 1019
start state: 826147305    goal state: 123456780    using h(2)
	Solution found:  No
	Total number of sta

start state: 326584107    goal state: 123456780    using h(2)
	Solution found:  No
	Total number of states explored: 2
	Time taken for execution : 0:00:00.000993

Retry Count: 1336
start state: 832047615    goal state: 123456780    using h(2)
	Solution found:  No
	Total number of states explored: 4
	Time taken for execution : 0:00:00.001277

Retry Count: 1337
start state: 651238470    goal state: 123456780    using h(2)
	Solution found:  No
	Total number of states explored: 1
	Time taken for execution : 0:00:00

Retry Count: 1338
start state: 427830156    goal state: 123456780    using h(2)
	Solution found:  No
	Total number of states explored: 2
	Time taken for execution : 0:00:00.000722

Retry Count: 1339
start state: 413875602    goal state: 123456780    using h(2)
	Solution found:  No
	Total number of states explored: 2
	Time taken for execution : 0:00:00

Retry Count: 1340
start state: 658437120    goal state: 123456780    using h(2)
	Solution found:  No
	Total number of states ex

...

start state: 401375862    goal state: 123456780    using h(2)
	Solution found:  No
	Total number of states explored: 2
	Time taken for execution : 0:00:00.002078

Retry Count: 5370
start state: 134057268    goal state: 123456780    using h(2)
	Solution found:  No
	Total number of states explored: 2
	Time taken for execution : 0:00:00.000988

Retry Count: 5371
start state: 825673104    goal state: 123456780    using h(2)
	Solution found:  No
	Total number of states explored: 2
	Time taken for execution : 0:00:00.001964

Retry Count: 5372
start state: 387410256    goal state: 123456780    using h(2)
	Solution found:  No
	Total number of states explored: 2
	Time taken for execution : 0:00:00.001881

Retry Count: 5373
start state: 274605138    goal state: 123456780    using h(2)
	Solution found:  No
	Total number of states explored: 2
	Time taken for execution : 0:00:00.001064

Retry Count: 5374
start state: 106753824    goal state: 123456780    using h(2)
	Solution found:  No
	Total numbe


Retry Count: 5526
start state: 136805472    goal state: 123456780    using h(2)
	Solution found:  No
	Total number of states explored: 3
	Time taken for execution : 0:00:00.000994

Retry Count: 5527
start state: 47613852    goal state: 123456780    using h(2)
	Solution found:  No
	Total number of states explored: 5
	Time taken for execution : 0:00:00.002983

Retry Count: 5528
start state: 325487160    goal state: 123456780    using h(2)
	Solution found:  No
	Total number of states explored: 1
	Time taken for execution : 0:00:00

Retry Count: 5529
start state: 605234781    goal state: 123456780    using h(2)
	Solution found:  No
	Total number of states explored: 2
	Time taken for execution : 0:00:00.001191

Retry Count: 5530
start state: 450367281    goal state: 123456780    using h(2)
	Solution found:  No
	Total number of states explored: 1
	Time taken for execution : 0:00:00

Retry Count: 5531
start state: 508712463    goal state: 123456780    using h(2)
	Solution found:  No
	Total n

start state: 638017245    goal state: 123456780    using h(2)
	Solution found:  No
	Total number of states explored: 4
	Time taken for execution : 0:00:00.004555

Retry Count: 5749
start state: 834652710    goal state: 123456780    using h(2)
	Solution found:  No
	Total number of states explored: 1
	Time taken for execution : 0:00:00.000990

Retry Count: 5750
start state: 506378142    goal state: 123456780    using h(2)
	Solution found:  No
	Total number of states explored: 1
	Time taken for execution : 0:00:00.001011

Retry Count: 5751
start state: 17265348    goal state: 123456780    using h(2)
	Solution found:  No
	Total number of states explored: 3
	Time taken for execution : 0:00:00.003189

Retry Count: 5752
start state: 427680513    goal state: 123456780    using h(2)
	Solution found:  No
	Total number of states explored: 2
	Time taken for execution : 0:00:00.000915

Retry Count: 5753
start state: 273184560    goal state: 123456780    using h(2)
	Solution found:  No
	Total number

start state: 504681372    goal state: 123456780    using h(2)
	Solution found:  No
	Total number of states explored: 4
	Time taken for execution : 0:00:00.002199

Retry Count: 5957
start state: 217684035    goal state: 123456780    using h(2)
	Solution found:  No
	Total number of states explored: 1
	Time taken for execution : 0:00:00

Retry Count: 5958
start state: 531467208    goal state: 123456780    using h(2)
	Solution found:  No
	Total number of states explored: 2
	Time taken for execution : 0:00:00.002069

Retry Count: 5959
start state: 518047632    goal state: 123456780    using h(2)
	Solution found:  No
	Total number of states explored: 4
	Time taken for execution : 0:00:00.002000

Retry Count: 5960
start state: 478261053    goal state: 123456780    using h(2)
	Solution found:  No
	Total number of states explored: 1
	Time taken for execution : 0:00:00

Retry Count: 5961
start state: 750824136    goal state: 123456780    using h(2)
	Solution found:  No
	Total number of states ex

start state: 128073645    goal state: 123456780    using h(2)
	Solution found:  No
	Total number of states explored: 4
	Time taken for execution : 0:00:00.003018

Retry Count: 6125
start state: 147250836    goal state: 123456780    using h(2)
	Solution found:  No
	Total number of states explored: 2
	Time taken for execution : 0:00:00.001000

Retry Count: 6126
start state: 741835620    goal state: 123456780    using h(2)
	Solution found:  No
	Total number of states explored: 1
	Time taken for execution : 0:00:00.001000

Retry Count: 6127
start state: 231548706    goal state: 123456780    using h(2)
	Solution found:  No
	Total number of states explored: 1
	Time taken for execution : 0:00:00.001000

Retry Count: 6128
start state: 241038675    goal state: 123456780    using h(2)
	Solution found:  No
	Total number of states explored: 4
	Time taken for execution : 0:00:00.003000

Retry Count: 6129
start state: 240157638    goal state: 123456780    using h(2)
	Solution found:  No
	Total numbe